In [1]:
import fdtd as oNN
import config as cfg
import torch
import numpy as np
from dataset import core_data_loader

In [2]:
def Init():
    oNN.set_backend(cfg.backend)
    
    grid = oNN.Grid(
        shape = (cfg.Nx, cfg.Ny, 1),
        grid_spacing=cfg.dx,
        permittivity=1.0
    )
    
    grid[0:10, :, :] = oNN.PML(name="pml_xlow")
    grid[-10:, :, :] = oNN.PML(name="pml_xhigh")
    grid[:, 0:10, :] = oNN.PML(name="pml_ylow")
    grid[:, -10:, :] = oNN.PML(name="pml_yhigh")
    
    for i in range(cfg.ports):
        grid[0:cfg.source_loc, cfg.ports_slice[i], 0] = oNN.Object(
            permittivity=2.8 * torch.ones([cfg.source_loc, cfg.port_width, 1]), name=f"wg{i}"
        )
        grid[cfg.detector_loc:, cfg.ports_slice[i], 0] = oNN.Object(
            permittivity=2.8 * torch.ones([cfg.Nx - cfg.detector_loc, cfg.port_width, 1]), name=f"op{i}"
        )
        grid[cfg.detector_loc, cfg.ports_slice[i], 0] = oNN.LineDetector(
            name=f"detector{i}"
        )
    return grid

In [3]:
def set_source(grid, source):
    for i in range(cfg.ports):
        grid[cfg.source_loc, cfg.ports_slice[i], 0] = oNN.LineSource(
            period=cfg.WAVELENGTH / cfg.SPEED_LIGHT, phase_shift=source[i] * torch.pi, name=f"source{i}"
        )
    return grid

In [4]:
def set_permittivity(grid, radius_matrix: np.ndarray):
    
    radius_matrix[radius_matrix < 0.3] = 0
    radius_matrix = torch.tensor(radius_matrix * 10)
    print(radius_matrix)
    
    x, y = torch.meshgrid(torch.arange(cfg.center_size), torch.arange(cfg.center_size), indexing='ij')
    
    outside_circle = torch.ones((cfg.center_size, cfg.center_size), dtype=torch.int)
    
    for cx, cy, radius in zip(cfg.x_centers, cfg.y_centers, radius_matrix.flatten()):
        mask = (x - cx) ** 2 + (y - cy) ** 2 <= radius ** 2
        outside_circle[mask] = 0
        
    permittivity = outside_circle.view(cfg.center_size, cfg.center_size, 1).float()
    grid[cfg.center_slice, cfg.center_slice, 0] = oNN.Object(
        permittivity=permittivity * 1.8 + 1, name="core"
    )
    return grid

`detectors[index].detector_values()`返回一个字典，包含两个键：`E`和`H`。

每个键下有一个长度为`timesteps`的列表，列表的元素是一个`torch.Tensor`，为对应的电场或磁场值。

该`torch.Tensor`的`shape`为`(len, 3)`。`len`为`LineDetector`中点的个数，`3`代表三个方向的分量。

In [7]:
train_loader, test_loader = core_data_loader(batch_size=1)

radius_matrix = np.random.rand(10, 10)

it = 0
for data, labels in train_loader:
    silicon_plane = Init()
    silicon_plane = set_source(silicon_plane, data[0])
    silicon_plane = set_permittivity(silicon_plane, radius_matrix)
    silicon_plane.run(cfg.time_step)
    # silicon_plane.visualize(z=0)
    output = torch.tensor([], device=cfg.device)
    output2 = []
    for i in range(cfg.ports): # Ez的值作为输出，空间上取平均，时间上取和
        detector_values_E = torch.stack(silicon_plane.detectors[i].detector_values()["E"], dim=0)
        output = torch.cat([output, torch.mean(detector_values_E[:,:,-1]**2, dim=1).sum(dim=0).unsqueeze(-1)])
        output2.append(torch.mean(detector_values_E[:,:,-1]**2, dim=1).sum(dim=0))
    print(output)
    print(output2)
    break
        
    it += 1
    if it == 30:
        break

Loading core data from F:\pyDistilledFDTD\dataset\.cache\core\greedy\pca-components-10\batch-size-1-eta-0.1.pkl
tensor([[9.2065, 7.3481, 9.2641, 0.0000, 5.4973, 0.0000, 0.0000, 7.4209, 7.4498,
         3.9711],
        [5.6725, 8.9796, 8.9534, 0.0000, 3.1007, 4.1009, 3.2809, 5.2186, 8.7539,
         0.0000],
        [7.5517, 6.6108, 4.6681, 4.7567, 8.5025, 3.3377, 0.0000, 8.9678, 6.1591,
         0.0000],
        [0.0000, 0.0000, 5.5745, 9.4741, 9.8627, 7.0700, 4.5575, 8.3655, 5.0552,
         0.0000],
        [4.7466, 3.8091, 0.0000, 0.0000, 0.0000, 4.5726, 0.0000, 9.1127, 3.3550,
         4.8871],
        [0.0000, 7.5348, 8.9828, 5.6578, 3.7893, 3.4213, 0.0000, 7.9684, 3.8677,
         8.9160],
        [5.4274, 5.7888, 8.4553, 0.0000, 6.6821, 8.3068, 5.9357, 0.0000, 3.5334,
         7.9497],
        [7.4626, 6.4905, 6.3554, 0.0000, 9.6978, 4.4577, 0.0000, 0.0000, 8.3626,
         0.0000],
        [0.0000, 0.0000, 7.4952, 8.5561, 8.2479, 4.0859, 9.9229, 9.5999, 7.3201,
         5.4796

100%|██████████| 1500/1500 [00:31<00:00, 47.26it/s]


tensor([0.5171, 0.3330, 0.1644, 0.6797, 0.3052, 0.2757, 0.2223, 0.1845, 0.5338,
        0.3501], device='cuda:0')
[tensor(0.5171, device='cuda:0'), tensor(0.3330, device='cuda:0'), tensor(0.1644, device='cuda:0'), tensor(0.6797, device='cuda:0'), tensor(0.3052, device='cuda:0'), tensor(0.2757, device='cuda:0'), tensor(0.2223, device='cuda:0'), tensor(0.1845, device='cuda:0'), tensor(0.5338, device='cuda:0'), tensor(0.3501, device='cuda:0')]
